In [164]:
df <- read.csv("training_data.csv", header = TRUE, sep = ",")

In [165]:
library(dummies)
library(stringdist)

In [166]:
lowercase_all = function(df) {
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "character") {
            df[,column] <<- tolower(df[,column])
        }
    }
}

In [167]:
lowercase_all(df)
df

id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
<int>,<dbl>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<chr>,<int>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
69572,6000,14/03/2011,roman,1390,roman,34.93809,-9.85632177,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0,06/03/2013,grumeti,1399,grumeti,34.69877,-2.14746569,zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25,25/02/2013,lottery club,686,world vision,37.46066,-3.82132853,kwa mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0,28/01/2013,unicef,263,unicef,38.48616,-11.15529772,zahanati ya nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0,13/07/2011,action in a,0,artisan,31.13085,-1.82535885,shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
9944,20,13/03/2011,mkinga distric coun,0,dwe,39.17280,-4.76558728,tajiri,0,...,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe
19816,0,01/10/2012,dwsp,0,dwsp,33.36241,-3.76636472,kwa ngomho,0,...,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
54551,0,09/10/2012,rwssp,0,dwe,32.62062,-4.22619802,tushirikiane,0,...,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
53934,0,03/11/2012,wateraid,0,water aid,32.71110,-5.14671181,kwa ramadhan musa,0,...,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump


In [168]:
#filter long/lat
x_up_right = 0.022
y_up_right = 40.729
x_down_left = -12.729
y_down_left = 28.138

df <- df[df$latitude > x_down_left & df$latitude < x_up_right & df$longitude > y_down_left & df$longitude < y_up_right, ] #filter pumps in atlantic fucking ocean

In [169]:
drops <- c("latitude", "longitude", "funder", "date_recorded", "gps_height", "wpt_name", "num_private", "lga", "ward", "subvillage", "region_code", "district_code", "recorded_by", "scheme_name", "waterpoint_type_group", "payment", "management", "management_group", "extraction_type_group", "extraction_type_class", "quantity_group", "quality_group", "source")
df = df[ , !(names(df) %in% drops)] #drop useless (a priori) features, maybe after feature selection remove less

In [170]:
df

,id,amount_tsh,installer,basin,region,population,public_meeting,scheme_management,permit,construction_year,extraction_type,payment_type,water_quality,quantity,source_type,source_class,waterpoint_type
,<int>,<dbl>,<chr>,<chr>,<chr>,<int>,<lgl>,<chr>,<lgl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,6000,roman,lake nyasa,iringa,109,TRUE,vwc,FALSE,1999,gravity,annually,soft,enough,spring,groundwater,communal standpipe
2,8776,0,grumeti,lake victoria,mara,280,NA,other,TRUE,2010,gravity,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe
3,34310,25,world vision,pangani,manyara,250,TRUE,vwc,TRUE,2009,gravity,per bucket,soft,enough,dam,surface,communal standpipe multiple
4,67743,0,unicef,ruvuma / southern coast,mtwara,58,TRUE,vwc,TRUE,1986,submersible,never pay,soft,dry,borehole,groundwater,communal standpipe multiple
5,19728,0,artisan,lake victoria,kagera,0,TRUE,,TRUE,0,gravity,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe
6,9944,20,dwe,pangani,tanga,1,TRUE,vwc,TRUE,2009,submersible,per bucket,salty,enough,other,unknown,communal standpipe multiple
7,19816,0,dwsp,internal,shinyanga,0,TRUE,vwc,TRUE,0,swn 80,never pay,soft,enough,borehole,groundwater,hand pump
8,54551,0,dwe,lake tanganyika,shinyanga,0,TRUE,,TRUE,0,nira/tanira,unknown,milky,enough,shallow well,groundwater,hand pump
9,53934,0,water aid,lake tanganyika,tabora,0,TRUE,vwc,TRUE,0,india mark ii,never pay,salty,seasonal,borehole,groundwater,hand pump


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"


In [172]:
similarity_test_apply = function(line) {
    #
    NAs = is.na(line)
    if(any(NAs) == TRUE) {
        idxs = which(is.na(line))
        #print(df[names(df)[idxs]])
        line2 = line[-idxs]
        most_similar = df2[which.min(Reduce(`+`,Map(stringdist, setdiff(line2, df2[, -idxs]), line2, method='jaccard'))),]
        #most_similar
        print(length(most_similar))
        print(length(line))
        line[idxs] <- most_similar[idxs]
        #print(line)
        print(which(df2$id == line$id))
    return(line)
        
    } 
}

In [173]:
#testing similarity

df2 = df[1:30,]
start.time = Sys.time()
df2 = apply(df2, 1, similarity_test_apply)
round(Sys.time() - start.time,5)
#df2

[1] 17
[1] 17
integer(0)


Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
"longer argument not a multiple of length of shorter"


[1] 17
[1] 17
[1] 19


Time difference of 0.02693 secs

In [ ]:
### DON'T RUN CELLS FROM DOWN HERE ###

In [78]:
work_on = setdiff(names(df), c("amount_tsh", "id"))
df[work_on][df[work_on] == 0 | df[work_on] == "-" | df[work_on] == ""] = NA

In [28]:
df <-  structure(list(Age = c(23, 29, 45, 24, 28), Occupation = c("Builder", 
"Worker", "Contractor", "Engineer", "Doctor"), Nationality = c("German", 
"British", "Vietnamese", "German", "Indian")), .Names = c("Age", 
"Occupation", "Nationality"), row.names = c(NA, -5L), class = "data.frame")

newdata <- list(23,"Doctor","German")

In [29]:
df[which.min(Reduce(`+`,Map(stringdist,df, newdata,method='jaccard'))),]

,Age,Occupation,Nationality
,<dbl>,<chr>,<chr>
1,23,Builder,German


[1] 40


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
print(df[1, "latitude"])
print(df[1, "longitude"])

[1] -9.856322
[1] 34.93809


In [18]:
df

,id,amount_tsh,installer,basin,region,population,public_meeting,scheme_management,permit,construction_year,extraction_type,payment_type,water_quality,quantity,source_type,source_class,waterpoint_type
,<int>,<dbl>,<chr>,<chr>,<chr>,<int>,<lgl>,<chr>,<lgl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,6000,Roman,Lake Nyasa,Iringa,109,TRUE,VWC,FALSE,1999,gravity,annually,soft,enough,spring,groundwater,communal standpipe
2,8776,0,GRUMETI,Lake Victoria,Mara,280,NA,Other,TRUE,2010,gravity,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe
3,34310,25,World vision,Pangani,Manyara,250,TRUE,VWC,TRUE,2009,gravity,per bucket,soft,enough,dam,surface,communal standpipe multiple
4,67743,0,UNICEF,Ruvuma / Southern Coast,Mtwara,58,TRUE,VWC,TRUE,1986,submersible,never pay,soft,dry,borehole,groundwater,communal standpipe multiple
5,19728,0,Artisan,Lake Victoria,Kagera,0,TRUE,,TRUE,0,gravity,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe
6,9944,20,DWE,Pangani,Tanga,1,TRUE,VWC,TRUE,2009,submersible,per bucket,salty,enough,other,unknown,communal standpipe multiple
7,19816,0,DWSP,Internal,Shinyanga,0,TRUE,VWC,TRUE,0,swn 80,never pay,soft,enough,borehole,groundwater,hand pump
8,54551,0,DWE,Lake Tanganyika,Shinyanga,0,TRUE,,TRUE,0,nira/tanira,unknown,milky,enough,shallow well,groundwater,hand pump
9,53934,0,Water Aid,Lake Tanganyika,Tabora,0,TRUE,VWC,TRUE,0,india mark ii,never pay,salty,seasonal,borehole,groundwater,hand pump


In [88]:
categories <- unique(df$installer) 

In [89]:
length(categories)
as.data.frame(table(df$installer))

#test[order(-test$Freq),]

[1] 2144

Var1,Freq
<fct>,<int>
A.D.B,1
AAR,4
Aartisa,1
ABASIA,29
ABD,1
ABDALA,1
Abdallah Ally Wazir,1
ABDUL,1
AC,2


In [ ]:
region, lga, ward, subvillage, basin, wpt_name, 

In [ ]:
upper left -0.6721467040207111, 28.299198544014903
upper right 0.022237, 40.729917
lower right -12.609672, 41.940606
lower left -12.729036 = x = latitude, 28.138472 = y = longitude


if (x > x1 and x < x2 and y > y1 and y < y2)

In [12]:
df$funder = tolower(df$funder)
df

,id,amount_tsh,funder,installer,basin,region,population,public_meeting,scheme_management,permit,construction_year,extraction_type,payment_type,water_quality,quantity,source_type,source_class,waterpoint_type
,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<lgl>,<chr>,<lgl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,6000,roman,Roman,Lake Nyasa,Iringa,109,TRUE,VWC,FALSE,1999,gravity,annually,soft,enough,spring,groundwater,communal standpipe
2,8776,0,grumeti,GRUMETI,Lake Victoria,Mara,280,NA,Other,TRUE,2010,gravity,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe
3,34310,25,lottery club,World vision,Pangani,Manyara,250,TRUE,VWC,TRUE,2009,gravity,per bucket,soft,enough,dam,surface,communal standpipe multiple
4,67743,0,unicef,UNICEF,Ruvuma / Southern Coast,Mtwara,58,TRUE,VWC,TRUE,1986,submersible,never pay,soft,dry,borehole,groundwater,communal standpipe multiple
5,19728,0,action in a,Artisan,Lake Victoria,Kagera,0,TRUE,,TRUE,0,gravity,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe
6,9944,20,mkinga distric coun,DWE,Pangani,Tanga,1,TRUE,VWC,TRUE,2009,submersible,per bucket,salty,enough,other,unknown,communal standpipe multiple
7,19816,0,dwsp,DWSP,Internal,Shinyanga,0,TRUE,VWC,TRUE,0,swn 80,never pay,soft,enough,borehole,groundwater,hand pump
8,54551,0,rwssp,DWE,Lake Tanganyika,Shinyanga,0,TRUE,,TRUE,0,nira/tanira,unknown,milky,enough,shallow well,groundwater,hand pump
9,53934,0,wateraid,Water Aid,Lake Tanganyika,Tabora,0,TRUE,VWC,TRUE,0,india mark ii,never pay,salty,seasonal,borehole,groundwater,hand pump


In [ ]:
missing_imputation_numbers = function(df) {
    #print(df)
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "integer" | columnClass == "numeric") {
            print(columnClass)
            df[is.na(df[,column]), column] <<- mean(df[,column], na.rm = TRUE)
        }
     }
}

In [4]:
missing_imputation_logical = function(df) {
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "logical") {
            categories <- unique(df[,column]) 
            df[is.na(df[,column]), column] <<- sample(categories, 1)
        }
    }
}

In [ ]:
missing_imputation_numbers = function(df) {
    #print(df)
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "integer" | columnClass == "numeric") {
            print(columnClass)
            df[is.na(df[,column]), column] <<- mean(df[,column], na.rm = TRUE)
        }
     }
}

In [44]:
#one-hot enc

dftest = df
keep = c("basin", "region")
print(names(dftest))
dftest_ = dummy.data.frame(dftest[,keep], sep="_")

 [1] "id"                "amount_tsh"        "installer"        
 [4] "basin"             "region"            "population"       
 [7] "public_meeting"    "scheme_management" "permit"           
[10] "construction_year" "extraction_type"   "payment_type"     
[13] "water_quality"     "quantity"          "source_type"      
[16] "source_class"      "waterpoint_type"  


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
